In [52]:
from collections import Counter 

import pandas as pd
import numpy as np
import re

In [53]:
data_path = "data/"
df = pd.read_csv(data_path + "cleaned_train.csv")

In [54]:
# df = df[:200]
df

,raw_address,POI/street
0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,setu siung 119 rt 5 1 13880 cipayung,/siung
2,"toko dita , kertosono",toko dita/
3,jl. orde baru,/jl. orde baru
4,"raya samb gede , 299 toko bb kids",toko bb kids/raya samb gede
...,...,...
265518,karawaci baru kakap raya 156 rt 1 rw 3 karawaci,/kakap raya
265519,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani
265520,"raya cila kko , cilandak timur kel.",/raya cila kko
265521,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/


In [55]:
def getPOI(s) :
    return s.split("/")[0]

def getStreet(s) :
    return s.split("/")[1]

df["POI"] = df["POI/street"].apply(getPOI)
df["Street"] = df["POI/street"].apply(getStreet) 

In [56]:
def create_label(s, poi, street) :
    tokens = s.split(" ")
    n = len(tokens)
    tp = poi.split(" ")
    ts = street.split(" ")
    tags = []
    i = 0
    while i < n :
        if len(tp) > 0 and tokens[i] == tp[0] :
            j = 1
            while j < len(tp) :
                if i+j < n and tokens[i+j] == tp[j]:
                    j += 1
                else :
                    break
            if j == len(tp) :
                tags.append("B-POI")
                for k in range(1, j) :
                    tags.append("I-POI")
                i += j
            else :
                tags.append("O")
                i += 1
        elif len(ts) > 0 and tokens[i] == ts[0] :
            j = 1
            while j < len(ts) :
                if i+j < n and tokens[i+j] == ts[j] :
                    j += 1
                else :
                    break
            if j == len(ts) :
                tags.append("B-STREET")
                for k in range(1, j) :
                    tags.append("I-STREET")
                i += j
            else :
                tags.append("O")
                i += 1                
        else :
            tags.append("O")
            i += 1
        
#         print(tokens)
#         print(tags)

    if tokens[-1] != "." :
        tokens.append(".")
        tags.append("O")
    
    res = ""
#     print(tokens)
#     print(tags)
    for token, tag in zip(tokens, tags) :
        res += "{}\t{}\n".format(token, tag)
    return res

In [57]:
s = "raya samb gede , 299 toko bb kids"
poi = "toko bb kids"
street = "raya samb gede"
res = create_label(s, poi, street)
print(res)

raya	B-STREET
samb	I-STREET
gede	I-STREET
,	O
299	O
toko	B-POI
bb	I-POI
kids	I-POI
.	O



In [58]:
df["Label"] = df.apply(lambda x: create_label(x.raw_address, x.POI, x.Street), axis=1)

In [59]:
df

,raw_address,POI/street,POI,Street,Label
0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,,jl kapuk timur delta sili iii lippo cika,jl\tB-STREET\nkapuk\tI-STREET\ntimur\tI-STREET...
1,setu siung 119 rt 5 1 13880 cipayung,/siung,,siung,setu\tO\nsiung\tB-STREET\n119\tO\nrt\tO\n5\tO\...
2,"toko dita , kertosono",toko dita/,toko dita,,"toko\tB-POI\ndita\tI-POI\n,\tO\nkertosono\tO\n..."
3,jl. orde baru,/jl. orde baru,,jl. orde baru,jl.\tB-STREET\norde\tI-STREET\nbaru\tI-STREET\...
4,"raya samb gede , 299 toko bb kids",toko bb kids/raya samb gede,toko bb kids,raya samb gede,raya\tB-STREET\nsamb\tI-STREET\ngede\tI-STREET...
...,...,...,...,...,...
265518,karawaci baru kakap raya 156 rt 1 rw 3 karawaci,/kakap raya,,kakap raya,karawaci\tO\nbaru\tO\nkakap\tB-STREET\nraya\tI...
265519,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani,,jend ahmad yani,jend\tB-STREET\nahmad\tI-STREET\nyani\tI-STREE...
265520,"raya cila kko , cilandak timur kel.",/raya cila kko,,raya cila kko,raya\tB-STREET\ncila\tI-STREET\nkko\tI-STREET\...
265521,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/,taman asri,,jalan\tO\ncipadu\tO\njaya\tO\ntaman\tB-POI\nas...


In [60]:
final = df.sample(frac=1, random_state=0).reset_index(drop=True)
final

,raw_address,POI/street,POI,Street,Label
0,"pademangan timur g 3 , no 37 rt 6 9 pademangan",/g 3,,g 3,pademangan\tO\ntimur\tO\ng\tB-STREET\n3\tI-STR...
1,"pt chubb general insurance indonesia , chubb s...",chubb square/jl. mh thamrin,chubb square,jl. mh thamrin,pt\tO\nchubb\tO\ngeneral\tO\ninsurance\tO\nind...
2,"pemakaman umum kedondong , kecapi mel , no b 1...",pemakaman umum kedondong/kecapi mel,pemakaman umum kedondong,kecapi mel,pemakaman\tB-POI\numum\tI-POI\nkedondong\tI-PO...
3,"got roy , pulo gebang",/got roy,,got roy,"got\tB-STREET\nroy\tI-STREET\n,\tO\npulo\tO\ng..."
4,"holycow steak house , tulip",holycow steak house/tulip,holycow steak house,tulip,"holycow\tB-POI\nsteak\tI-POI\nhouse\tI-POI\n,\..."
...,...,...,...,...,...
265518,"toko minasa te'ne , bonto raya , tamalanrea",toko minasa te'ne/bonto raya,toko minasa te'ne,bonto raya,"toko\tB-POI\nminasa\tI-POI\nte'ne\tI-POI\n,\tO..."
265519,perumahan wahana cikarang blok 17 kelurahan : ...,perumahan wahana/,perumahan wahana,,perumahan\tB-POI\nwahana\tI-POI\ncikarang\tO\n...
265520,"perumahan rachita garden , jl. poros kariango ...",perumahan rachita garden/jl. poros kariango,perumahan rachita garden,jl. poros kariango,perumahan\tB-POI\nrachita\tI-POI\ngarden\tI-PO...
265521,jln sematang borang komplek griya cipta persad...,komplek griya cipta persada/jln sematang borang,komplek griya cipta persada,jln sematang borang,jln\tB-STREET\nsematang\tI-STREET\nborang\tI-S...


In [61]:
n = len(final)
nt = int(0.95 * n)
train = final[:nt+1]
test = final[nt:]

In [62]:
train

,raw_address,POI/street,POI,Street,Label
0,"pademangan timur g 3 , no 37 rt 6 9 pademangan",/g 3,,g 3,pademangan\tO\ntimur\tO\ng\tB-STREET\n3\tI-STR...
1,"pt chubb general insurance indonesia , chubb s...",chubb square/jl. mh thamrin,chubb square,jl. mh thamrin,pt\tO\nchubb\tO\ngeneral\tO\ninsurance\tO\nind...
2,"pemakaman umum kedondong , kecapi mel , no b 1...",pemakaman umum kedondong/kecapi mel,pemakaman umum kedondong,kecapi mel,pemakaman\tB-POI\numum\tI-POI\nkedondong\tI-PO...
3,"got roy , pulo gebang",/got roy,,got roy,"got\tB-STREET\nroy\tI-STREET\n,\tO\npulo\tO\ng..."
4,"holycow steak house , tulip",holycow steak house/tulip,holycow steak house,tulip,"holycow\tB-POI\nsteak\tI-POI\nhouse\tI-POI\n,\..."
...,...,...,...,...,...
252242,"kopi laka laka , jend ahmad yani , no 333 17141",kopi laka laka/jend ahmad yani,kopi laka laka,jend ahmad yani,"kopi\tB-POI\nlaka\tI-POI\nlaka\tI-POI\n,\tO\nj..."
252243,pasar lama jl kisa sukasari tangerang,pasar lama/jl kisa,pasar lama,jl kisa,pasar\tB-POI\nlama\tI-POI\njl\tB-STREET\nkisa\...
252244,"persa 1 , no 83 cimanggis",/persa 1,,persa 1,"persa\tB-STREET\n1\tI-STREET\n,\tO\nno\tO\n83\..."
252245,marz 33 limo,/marz,,marz,marz\tB-STREET\n33\tO\nlimo\tO\n.\tO\n


In [63]:
def save_file(filename, arr) :
    f = open(filename, "w+",  encoding='utf-8')
    for s in arr :
        f.write(s.encode("ascii", "ignore").decode())
        f.write("\n")
    f.close()

In [64]:
train_path = data_path + "bert-fine-tune/train.txt"
validation_path = data_path + "bert-fine-tune/validation.txt"

train_data = train["Label"].values.tolist()
test_data = test["Label"].values.tolist()


ntry = None
# ntry = 1000
if ntry :
    train_data = train_data[:ntry]
    test_data = test_data[:ntry]

save_file(train_path, train_data)
save_file(validation_path, test_data)